In [1]:
import numpy as np
import pandas as pd
import bokeh.plotting as bp
from bokeh.layouts import gridplot
#bp.output_notebook()

se graficarán los resultados que se generan con los distintos modelos de clasificación utilizados, no se tendrá en cuenta el clasificador GaussianNB ya que este no arrojó buenos resultados

In [2]:
result_KNC = pd.read_csv('../resultados/TrueFX/EUR-USD/KNeighborsClassfier/pnl_KNC_wsize3_2017-09.csv',
                        parse_dates=True, infer_datetime_format=True, index_col=0)
result_DTC = pd.read_csv('../resultados/TrueFX/EUR-USD/DecisionTreeClassifier/pnl_DTC_wsize3_2017-09.csv',
                        parse_dates=True, infer_datetime_format=True, index_col=0)
result_RFC = pd.read_csv('../resultados/TrueFX/EUR-USD/RandomForestClassifier/pnl_RFC_wsize3_2017-09.csv',
                        parse_dates=True, infer_datetime_format=True, index_col=0)

pintamos los resultados en conjunto para poder compararlos según su modelo...

In [3]:
result_DTC.head()

,PNL,buys,sells,longest_buys,longest_sells
2018-01-05,-0.23055,3084.0,3374.0,11.0,18.0
2018-01-08,-0.22908,5164.0,1350.0,3067.0,11.0
2018-01-09,-0.21093,5365.0,1290.0,4206.0,17.0
2018-01-10,-0.25554,3279.0,3741.0,15.0,49.0
2018-01-11,-0.24436,3368.0,3744.0,16.0,15.0


In [4]:
plot_1 = bp.figure(width=850, height=300,title='Profit and Loss',x_axis_type='datetime')
#es el mismo indice para todos entonces no hay problema
x = result_RFC.index
plot_1.line(x, result_RFC.iloc[:,0], color='blue', legend='RFC', alpha=0.5)
plot_1.circle(x, result_RFC.iloc[:,0], color='blue', alpha=0.5,size=3)
plot_1.line(x, result_DTC.iloc[:,0], color='red', legend='DTC', alpha=0.5)
plot_1.circle(x, result_DTC.iloc[:,0], color='red',alpha=0.5,size=3)
plot_1.line(x, result_KNC.iloc[:,0], color='green', legend='KNC', alpha=0.5)
plot_1.circle(x, result_KNC.iloc[:,0], color='green',alpha=0.5,size=3)
plot_1.line(x, np.zeros(len(result_RFC)), color='black', legend='PNL = 0', line_dash='dotted')
plot_1.xaxis.axis_label = 'datetime'
plot_1.yaxis.axis_label = 'PNL'
plot_1.legend.location ='bottom_center'
plot_1.legend.orientation = 'horizontal'

plot_2 = bp.figure(width=550, height=250, title='buys per day', x_axis_type='datetime')
plot_2.line(x, result_RFC.iloc[:,1], color='blue')
plot_2.line(x, result_DTC.iloc[:,1], color='red')
plot_2.line(x, result_KNC.iloc[:,1], color='green')
#p2_RFC.circle(x, result_RFC.iloc[:,1], color='green')
plot_2.xaxis.axis_label = 'datetime'
plot_2.yaxis.axis_label = 'buys per day'

#se utiliza el x_range para correlacionar las dos gráficas por el eje x
plot_3 = bp.figure(width=550, height=250, title='sells per day', x_axis_type='datetime',x_range=plot_2.x_range)
plot_3.line(x, result_RFC.iloc[:,2], color='blue')
plot_3.line(x, result_DTC.iloc[:,2], color='red')
plot_3.line(x, result_KNC.iloc[:,2], color='green')
#p3_RFC.circle(x, result_RFC.iloc[:,2], color='red')
plot_3.xaxis.axis_label = 'datetime'
plot_3.yaxis.axis_label = 'sells per day'

#para pintar el PNL que se va acumulando del KNC
PNL_accu_KNC = []
count_KNC = 0
for i in result_KNC.iloc[:,0]:
    count_KNC += i
    PNL_accu_KNC.append(count_KNC)
#para pintar el PNL que se va acumulando del DTC
PNL_accu_DTC = []
count_DTC = 0
for i in result_DTC.iloc[:,0]:
    count_DTC += i
    PNL_accu_DTC.append(count_DTC) 
#para pintar el PNL que se va acumulando del RFC
PNL_accu_RFC = []
count_RFC = 0
for i in result_RFC.iloc[:,0]:
    count_RFC += i
    PNL_accu_RFC.append(count_RFC)
    
plot_4 = bp.figure(width=850, height=300,title='PNL accumulated',x_axis_type='datetime',x_range=plot_1.x_range)
plot_4.line(x, PNL_accu_RFC,color='blue',legend='RFC',alpha=0.5)
plot_4.circle(x, PNL_accu_RFC, color='blue', alpha=0.5, size=3)
plot_4.line(x, PNL_accu_DTC,color='red',legend='DTC',alpha=0.5)
plot_4.circle(x, PNL_accu_DTC, color='red', alpha=0.5, size=3)
plot_4.line(x, PNL_accu_KNC,color='green',legend='KNC',alpha=0.5)
plot_4.circle(x, PNL_accu_KNC, color='green', alpha=0.5, size=3)
plot_4.line(x, np.zeros(len(result_RFC)), color='black', legend='PNL = 0', line_dash='dotted')
plot_4.xaxis.axis_label = 'datetime'
plot_4.yaxis.axis_label = 'PNL accumulated'
plot_4.legend.location ='bottom_center'
plot_4.legend.orientation = 'horizontal'

bp.show(gridplot([[plot_1],[plot_4],[plot_2,plot_3]], toolbar_location="right"))
bp.output_file(filename='../resultados/TrueFX/EUR-USD/grafico comparación.html', title='comparación')